# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [16]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


我们首先定义模型的输入与输出(训练数据)

In [ ]:
import numpy as np
import torch

NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

然后我们用PyTorch定义模型

In [17]:
# Define the model
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [18]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

以下是模型的训练代码

In [19]:
# Start training it
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]

        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find loss on training data
    loss = loss_fn(model(trX), trY).item()
    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1.2172406911849976
Epoch: 1 Loss: 1.1681287288665771
Epoch: 2 Loss: 1.1549371480941772
Epoch: 3 Loss: 1.1497281789779663
Epoch: 4 Loss: 1.1470859050750732
Epoch: 5 Loss: 1.1455236673355103
Epoch: 6 Loss: 1.1444897651672363
Epoch: 7 Loss: 1.1437405347824097
Epoch: 8 Loss: 1.1431617736816406
Epoch: 9 Loss: 1.1426900625228882
Epoch: 10 Loss: 1.1422890424728394
Epoch: 11 Loss: 1.1419371366500854
Epoch: 12 Loss: 1.1416198015213013
Epoch: 13 Loss: 1.141330599784851
Epoch: 14 Loss: 1.141067624092102
Epoch: 15 Loss: 1.1408230066299438
Epoch: 16 Loss: 1.1405951976776123
Epoch: 17 Loss: 1.1403796672821045
Epoch: 18 Loss: 1.1401728391647339
Epoch: 19 Loss: 1.1399768590927124
Epoch: 20 Loss: 1.1397902965545654
Epoch: 21 Loss: 1.1396117210388184
Epoch: 22 Loss: 1.1394388675689697
Epoch: 23 Loss: 1.1392732858657837
Epoch: 24 Loss: 1.139114260673523
Epoch: 25 Loss: 1.138958215713501
Epoch: 26 Loss: 1.1388083696365356
Epoch: 27 Loss: 1.138662576675415
Epoch: 28 Loss: 1.1385197639465332


Epoch: 241 Loss: 1.1044905185699463
Epoch: 242 Loss: 1.1039113998413086
Epoch: 243 Loss: 1.1035758256912231
Epoch: 244 Loss: 1.1031274795532227
Epoch: 245 Loss: 1.102744698524475
Epoch: 246 Loss: 1.1026242971420288
Epoch: 247 Loss: 1.1023188829421997
Epoch: 248 Loss: 1.1018552780151367
Epoch: 249 Loss: 1.1014649868011475
Epoch: 250 Loss: 1.1008750200271606
Epoch: 251 Loss: 1.1005561351776123
Epoch: 252 Loss: 1.1001912355422974
Epoch: 253 Loss: 1.0998426675796509
Epoch: 254 Loss: 1.0993032455444336
Epoch: 255 Loss: 1.099241852760315
Epoch: 256 Loss: 1.0986937284469604
Epoch: 257 Loss: 1.0981847047805786
Epoch: 258 Loss: 1.0977647304534912
Epoch: 259 Loss: 1.0974912643432617
Epoch: 260 Loss: 1.0967620611190796
Epoch: 261 Loss: 1.0965226888656616
Epoch: 262 Loss: 1.095814824104309
Epoch: 263 Loss: 1.0959606170654297
Epoch: 264 Loss: 1.0953201055526733
Epoch: 265 Loss: 1.0948870182037354
Epoch: 266 Loss: 1.0950857400894165
Epoch: 267 Loss: 1.0938347578048706
Epoch: 268 Loss: 1.093930602073

Epoch: 499 Loss: 0.9322087168693542
Epoch: 500 Loss: 0.9332177639007568
Epoch: 501 Loss: 0.9301274418830872
Epoch: 502 Loss: 0.9317154884338379
Epoch: 503 Loss: 0.9273042678833008
Epoch: 504 Loss: 0.9298312664031982
Epoch: 505 Loss: 0.9275972247123718
Epoch: 506 Loss: 0.9277393221855164
Epoch: 507 Loss: 0.9247757196426392
Epoch: 508 Loss: 0.9265440702438354
Epoch: 509 Loss: 0.9220213890075684
Epoch: 510 Loss: 0.9238983392715454
Epoch: 511 Loss: 0.9210586547851562
Epoch: 512 Loss: 0.9223806858062744
Epoch: 513 Loss: 0.9178904294967651
Epoch: 514 Loss: 0.9216134548187256
Epoch: 515 Loss: 0.9159965515136719
Epoch: 516 Loss: 0.918282151222229
Epoch: 517 Loss: 0.9150093197822571
Epoch: 518 Loss: 0.9165747165679932
Epoch: 519 Loss: 0.9124729633331299
Epoch: 520 Loss: 0.9145289063453674
Epoch: 521 Loss: 0.9118285775184631
Epoch: 522 Loss: 0.9129870533943176
Epoch: 523 Loss: 0.9081150889396667
Epoch: 524 Loss: 0.9117816686630249
Epoch: 525 Loss: 0.9078242778778076
Epoch: 526 Loss: 0.9086933135

Epoch: 767 Loss: 0.6435044407844543
Epoch: 768 Loss: 0.63999342918396
Epoch: 769 Loss: 0.6387038230895996
Epoch: 770 Loss: 0.6368626952171326
Epoch: 771 Loss: 0.6386045813560486
Epoch: 772 Loss: 0.638329267501831
Epoch: 773 Loss: 0.6330574750900269
Epoch: 774 Loss: 0.6348878145217896
Epoch: 775 Loss: 0.6332709789276123
Epoch: 776 Loss: 0.632623553276062
Epoch: 777 Loss: 0.6284645199775696
Epoch: 778 Loss: 0.6309924721717834
Epoch: 779 Loss: 0.6242780685424805
Epoch: 780 Loss: 0.6244730949401855
Epoch: 781 Loss: 0.6291320323944092
Epoch: 782 Loss: 0.62169349193573
Epoch: 783 Loss: 0.6228833794593811
Epoch: 784 Loss: 0.6191036701202393
Epoch: 785 Loss: 0.623494029045105
Epoch: 786 Loss: 0.6175232529640198
Epoch: 787 Loss: 0.6163040399551392
Epoch: 788 Loss: 0.6178536415100098
Epoch: 789 Loss: 0.6151330471038818
Epoch: 790 Loss: 0.6168912649154663
Epoch: 791 Loss: 0.611665666103363
Epoch: 792 Loss: 0.6143817901611328
Epoch: 793 Loss: 0.6136386394500732
Epoch: 794 Loss: 0.6088814735412598


Epoch: 1017 Loss: 0.39443856477737427
Epoch: 1018 Loss: 0.3934824764728546
Epoch: 1019 Loss: 0.3947215974330902
Epoch: 1020 Loss: 0.39331328868865967
Epoch: 1021 Loss: 0.3912915289402008
Epoch: 1022 Loss: 0.3913138210773468
Epoch: 1023 Loss: 0.39139053225517273
Epoch: 1024 Loss: 0.38899266719818115
Epoch: 1025 Loss: 0.39010369777679443
Epoch: 1026 Loss: 0.388510525226593
Epoch: 1027 Loss: 0.38725849986076355
Epoch: 1028 Loss: 0.38703039288520813
Epoch: 1029 Loss: 0.386525958776474
Epoch: 1030 Loss: 0.3854270875453949
Epoch: 1031 Loss: 0.38453006744384766
Epoch: 1032 Loss: 0.38470569252967834
Epoch: 1033 Loss: 0.38389238715171814
Epoch: 1034 Loss: 0.3822349011898041
Epoch: 1035 Loss: 0.38422515988349915
Epoch: 1036 Loss: 0.3812806308269501
Epoch: 1037 Loss: 0.38004738092422485
Epoch: 1038 Loss: 0.38008973002433777
Epoch: 1039 Loss: 0.37863266468048096
Epoch: 1040 Loss: 0.37881743907928467
Epoch: 1041 Loss: 0.3775920867919922
Epoch: 1042 Loss: 0.3774804174900055
Epoch: 1043 Loss: 0.37819

...

Epoch: 9032 Loss: 0.008466586470603943
Epoch: 9033 Loss: 0.008462836034595966
Epoch: 9034 Loss: 0.008463013917207718
Epoch: 9035 Loss: 0.008459692820906639
Epoch: 9036 Loss: 0.008459791541099548
Epoch: 9037 Loss: 0.008457338437438011
Epoch: 9038 Loss: 0.008457805030047894
Epoch: 9039 Loss: 0.008453432470560074
Epoch: 9040 Loss: 0.008452367037534714
Epoch: 9041 Loss: 0.008450603112578392
Epoch: 9042 Loss: 0.008451426401734352
Epoch: 9043 Loss: 0.008447221480309963
Epoch: 9044 Loss: 0.008447066880762577
Epoch: 9045 Loss: 0.008444958366453648
Epoch: 9046 Loss: 0.008444393053650856
Epoch: 9047 Loss: 0.008441461250185966
Epoch: 9048 Loss: 0.008440442383289337
Epoch: 9049 Loss: 0.00843806378543377
Epoch: 9050 Loss: 0.008440084755420685
Epoch: 9051 Loss: 0.008434836752712727
Epoch: 9052 Loss: 0.008435146883130074
Epoch: 9053 Loss: 0.008431784808635712
Epoch: 9054 Loss: 0.008431090973317623
Epoch: 9055 Loss: 0.008428198285400867
Epoch: 9056 Loss: 0.008428738452494144
Epoch: 9057 Loss: 0.008425

Epoch: 9279 Loss: 0.008099563419818878
Epoch: 9280 Loss: 0.00809687189757824
Epoch: 9281 Loss: 0.008099120110273361
Epoch: 9282 Loss: 0.008095290511846542
Epoch: 9283 Loss: 0.008095026016235352
Epoch: 9284 Loss: 0.008092232048511505
Epoch: 9285 Loss: 0.008091257885098457
Epoch: 9286 Loss: 0.008089445531368256
Epoch: 9287 Loss: 0.008089671842753887
Epoch: 9288 Loss: 0.008086046203970909
Epoch: 9289 Loss: 0.008085649460554123
Epoch: 9290 Loss: 0.008083582855761051
Epoch: 9291 Loss: 0.008084643632173538
Epoch: 9292 Loss: 0.008080287836492062
Epoch: 9293 Loss: 0.008080783300101757
Epoch: 9294 Loss: 0.008076895028352737
Epoch: 9295 Loss: 0.008078877814114094
Epoch: 9296 Loss: 0.008074348792433739
Epoch: 9297 Loss: 0.008075140416622162
Epoch: 9298 Loss: 0.008071615360677242
Epoch: 9299 Loss: 0.008071252144873142
Epoch: 9300 Loss: 0.008070751093327999
Epoch: 9301 Loss: 0.008067242801189423
Epoch: 9302 Loss: 0.008067724294960499
Epoch: 9303 Loss: 0.008064773865044117
Epoch: 9304 Loss: 0.008064

Epoch: 9503 Loss: 0.007791380397975445
Epoch: 9504 Loss: 0.007789172697812319
Epoch: 9505 Loss: 0.007789109833538532
Epoch: 9506 Loss: 0.007787961512804031
Epoch: 9507 Loss: 0.0077866557985544205
Epoch: 9508 Loss: 0.007786630652844906
Epoch: 9509 Loss: 0.007783663459122181
Epoch: 9510 Loss: 0.007784045301377773
Epoch: 9511 Loss: 0.007780935615301132
Epoch: 9512 Loss: 0.007780915126204491
Epoch: 9513 Loss: 0.0077777826227247715
Epoch: 9514 Loss: 0.007778650615364313
Epoch: 9515 Loss: 0.007775277830660343
Epoch: 9516 Loss: 0.007775663398206234
Epoch: 9517 Loss: 0.0077720643021166325
Epoch: 9518 Loss: 0.007773237768560648
Epoch: 9519 Loss: 0.007770067546516657
Epoch: 9520 Loss: 0.007770282216370106
Epoch: 9521 Loss: 0.007767021656036377
Epoch: 9522 Loss: 0.00776782026514411
Epoch: 9523 Loss: 0.007764363195747137
Epoch: 9524 Loss: 0.007764579262584448
Epoch: 9525 Loss: 0.007761350832879543
Epoch: 9526 Loss: 0.007763484492897987
Epoch: 9527 Loss: 0.007759047206491232
Epoch: 9528 Loss: 0.007

Epoch: 9741 Loss: 0.007485908921808004
Epoch: 9742 Loss: 0.007487741764634848
Epoch: 9743 Loss: 0.007485820911824703
Epoch: 9744 Loss: 0.0074849724769592285
Epoch: 9745 Loss: 0.007482319138944149
Epoch: 9746 Loss: 0.0074822669848799706
Epoch: 9747 Loss: 0.007479987572878599
Epoch: 9748 Loss: 0.007480609230697155
Epoch: 9749 Loss: 0.0074761779978871346
Epoch: 9750 Loss: 0.007477293722331524
Epoch: 9751 Loss: 0.007474077865481377
Epoch: 9752 Loss: 0.007473524194210768
Epoch: 9753 Loss: 0.007473571691662073
Epoch: 9754 Loss: 0.00747064920142293
Epoch: 9755 Loss: 0.007471099030226469
Epoch: 9756 Loss: 0.007468314375728369
Epoch: 9757 Loss: 0.007469233125448227
Epoch: 9758 Loss: 0.007466175593435764
Epoch: 9759 Loss: 0.007466055918484926
Epoch: 9760 Loss: 0.007464361377060413
Epoch: 9761 Loss: 0.007462109439074993
Epoch: 9762 Loss: 0.007462223991751671
Epoch: 9763 Loss: 0.007459437474608421
Epoch: 9764 Loss: 0.007460377179086208
Epoch: 9765 Loss: 0.0074572814628481865
Epoch: 9766 Loss: 0.00

Epoch: 9952 Loss: 0.007234291173517704
Epoch: 9953 Loss: 0.007235421333462
Epoch: 9954 Loss: 0.00723215052857995
Epoch: 9955 Loss: 0.007232627831399441
Epoch: 9956 Loss: 0.007229790091514587
Epoch: 9957 Loss: 0.0072288839146494865
Epoch: 9958 Loss: 0.007227601483464241
Epoch: 9959 Loss: 0.007228099275380373
Epoch: 9960 Loss: 0.007225700654089451
Epoch: 9961 Loss: 0.007226031739264727
Epoch: 9962 Loss: 0.007223958615213633
Epoch: 9963 Loss: 0.007222803775221109
Epoch: 9964 Loss: 0.007220800966024399
Epoch: 9965 Loss: 0.00721993250772357
Epoch: 9966 Loss: 0.0072196354158222675
Epoch: 9967 Loss: 0.007216942962259054
Epoch: 9968 Loss: 0.007217903155833483
Epoch: 9969 Loss: 0.007215120829641819
Epoch: 9970 Loss: 0.007216301281005144
Epoch: 9971 Loss: 0.007212281692773104
Epoch: 9972 Loss: 0.00721323024481535
Epoch: 9973 Loss: 0.007211113348603249
Epoch: 9974 Loss: 0.007209348026663065
Epoch: 9975 Loss: 0.0072080516256392
Epoch: 9976 Loss: 0.007207706570625305
Epoch: 9977 Loss: 0.00720587326

最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [20]:
# Output now
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', '12', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', '42', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', 'buzz', 'fizz', '70', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [29]:
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

96


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])